<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import keras
from keras.datasets import fashion_mnist
from PIL import Image
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

wandb.init(project="assignment-1", entity="swathi")

classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

def load_data():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  return {
      "x_train": x_train,
      "y_train": y_train,
      "x_test": x_test,
      "y_test": y_test
  }

def log_images():
  dataset = load_data()
  train_images = dataset["x_train"]
  train_labels = dataset["y_train"]
  images = []
  labels = []
  for i in range(len(train_images)):
    if len(labels)==len(classes):
      break
    if classes[train_labels[i]] not in labels:
      images.append(train_images[i])
      labels.append(classes[train_labels[i]])
  wandb.log({"Examples": [wandb.Image(img, caption=caption) for img,caption in zip(images,labels)]})

def dataset():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  return{
      'train':{
          'x': x_train.reshape([60000,784]),
          'y': y_train.reshape([60000,1])
      },
      'test':{
          'x': x_test.reshape([10000,784]),
          'y': y_test.reshape([10000,1])
      }
  }

load_data()
dataset()
log_images()

_runtime,4
_timestamp,1614727702
_step,0


_runtime,▁
_timestamp,▁
_step,▁
